# Fraud Detection

Classify transactions as fraud or non-fraud using an LSTM-based neural network. Data coming from https://github.com/IBM/TabFormer/tree/main/data/credit_card

## Authors

Natalie Jann [natalie.jann@ibm.com](mailto:natalie.jann@ibm.com)

Sebastian Lehrig [sebastian.lehrig1@ibm.com](mailto:sebastian.lehrig1@ibm.com)

Marvin Giessing [MARVING@de.ibm.com](mailto:MARVING@de.ibm.com)

## License

Apache-2.0 License

## 1.) Preprocess the dataset

In [1]:
import math
import numpy as np
import pandas as pd

from requests import get
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer, MinMaxScaler, LabelBinarizer
from sklearn_pandas import DataFrameMapper

def timeEncoder(X):
    X_hm = X['Time'].str.split(':', expand=True)
    d = pd.to_datetime(dict(
        year=X['Year'],
        month=X['Month'],
        day=X['Day'],
        hour=X_hm[0],
        minute=X_hm[1]
    )).astype(int)
    return pd.DataFrame(d)

def amtEncoder(X):
    amt = X.apply(lambda x: x[1:]).astype(float).map(lambda amt: max(1,amt)).map(math.log)
    return pd.DataFrame(amt)

def decimalEncoder(X, length=5):
    dnew = pd.DataFrame()
    for i in range(length):
        dnew[i] = np.mod(X, 10)
        X = np.floor_divide(X, 10)
    return dnew

def fraudEncoder(X):
    return np.where(X == 'Yes', 1, 0).astype(int)

f = get('https://ibm.box.com/shared/static/wamc5d0yve71jm46ntdp23q6xc001ew2.csv')
with open('card_transaction.v1.csv', 'wb') as fd:
    fd.write(f.content)

tdf = pd.read_csv('card_transaction.v1.csv', nrows=1_000_000)
tdf['Merchant Name'] = tdf['Merchant Name'].astype(str)
tdf.drop(['MCC', 'Zip', 'Merchant State'], axis=1, inplace=True)
tdf.sort_values(by=['User','Card'], inplace=True)
tdf.reset_index(inplace=True, drop=True)

mapper = DataFrameMapper([('Is Fraud?', FunctionTransformer(fraudEncoder)),
                      ('Merchant Name', [LabelEncoder(), FunctionTransformer(decimalEncoder), OneHotEncoder()]),
                      ('Merchant City', [LabelEncoder(), FunctionTransformer(decimalEncoder), OneHotEncoder()]),
                      (['Use Chip'], [SimpleImputer(strategy='constant'), LabelBinarizer()]),
                      (['Errors?'], [SimpleImputer(strategy='constant'), LabelBinarizer()]),
                      (['Year','Month','Day','Time'], [FunctionTransformer(timeEncoder), MinMaxScaler()]),
                      ('Amount', [FunctionTransformer(amtEncoder), MinMaxScaler()])
                     ], input_df=True, df_out=True)
mapper.fit(tdf)
tdf = mapper.transform(tdf)

tdf.head()

/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Is Fraud?,Merchant Name_x0_0,Merchant Name_x0_1,Merchant Name_x0_2,Merchant Name_x0_3,Merchant Name_x0_4,Merchant Name_x0_5,Merchant Name_x0_6,Merchant Name_x0_7,Merchant Name_x0_8,...,"Errors?_Bad PIN,","Errors?_Bad PIN,Insufficient Balance,","Errors?_Bad PIN,Technical Glitch,","Errors?_Bad Zipcode,","Errors?_Insufficient Balance,","Errors?_Insufficient Balance,Technical Glitch,","Errors?_Technical Glitch,",Errors?_missing_value,Year_Month_Day_Time,Amount
0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.268375,0.554906
1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.268377,0.413490
2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.268490,0.542650
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0.268544,0.550478
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.268604,0.526890


In [ ]:
tdf.to_csv('preprocessed_transactions.csv', index=False)

### Define the columns to be created in the database table

In [2]:
columns = ['Is Fraud?', 'Merchant Name_x0_0', 'Merchant Name_x0_1', 'Merchant Name_x0_2', 'Merchant Name_x0_3', 
            'Merchant Name_x0_4', 'Merchant Name_x0_5', 'Merchant Name_x0_6', 'Merchant Name_x0_7', 'Merchant Name_x0_8',
            'Merchant Name_x0_9', 'Merchant Name_x1_0', 'Merchant Name_x1_1', 'Merchant Name_x1_2', 'Merchant Name_x1_3',
            'Merchant Name_x1_4', 'Merchant Name_x1_5', 'Merchant Name_x1_6', 'Merchant Name_x1_7', 'Merchant Name_x1_8',
            'Merchant Name_x1_9', 'Merchant Name_x2_0', 'Merchant Name_x2_1', 'Merchant Name_x2_2', 'Merchant Name_x2_3',
            'Merchant Name_x2_4', 'Merchant Name_x2_5', 'Merchant Name_x2_6', 'Merchant Name_x2_7', 'Merchant Name_x2_8',
            'Merchant Name_x2_9', 'Merchant Name_x3_0', 'Merchant Name_x3_1', 'Merchant Name_x3_2', 'Merchant Name_x3_3',
            'Merchant Name_x3_4', 'Merchant Name_x3_5', 'Merchant Name_x3_6', 'Merchant Name_x3_7', 'Merchant Name_x3_8',
            'Merchant Name_x3_9', 'Merchant Name_x4_0', 'Merchant Name_x4_1', 'Merchant City_x0_0', 'Merchant City_x0_1',
            'Merchant City_x0_2', 'Merchant City_x0_3', 'Merchant City_x0_4', 'Merchant City_x0_5', 'Merchant City_x0_6',
            'Merchant City_x0_7', 'Merchant City_x0_8', 'Merchant City_x0_9', 'Merchant City_x1_0', 'Merchant City_x1_1',
            'Merchant City_x1_2', 'Merchant City_x1_3', 'Merchant City_x1_4', 'Merchant City_x1_5', 'Merchant City_x1_6',
            'Merchant City_x1_7', 'Merchant City_x1_8', 'Merchant City_x1_9', 'Merchant City_x2_0', 'Merchant City_x2_1',
            'Merchant City_x2_2', 'Merchant City_x2_3', 'Merchant City_x2_4', 'Merchant City_x2_5', 'Merchant City_x2_6',
            'Merchant City_x2_7', 'Merchant City_x2_8', 'Merchant City_x2_9', 'Merchant City_x3_0', 'Merchant City_x3_1',
            'Merchant City_x3_2', 'Merchant City_x3_3', 'Merchant City_x3_4', 'Merchant City_x4_0', 'Use Chip_Chip Transaction',
            'Use Chip_Online Transaction', 'Use Chip_Swipe Transaction', 'Errors?_Bad CVV,', 
            'Errors?_Bad CVV,Insufficient Balance,', 'Errors?_Bad CVV,Technical Glitch,', 'Errors?_Bad Card Number,', 
            'Errors?_Bad Card Number,Bad CVV,', 'Errors?_Bad Card Number,Bad Expiration,', 
            'Errors?_Bad Card Number,Bad Expiration,Technical Glitch,', 'Errors?_Bad Card Number,Insufficient Balance,', 
            'Errors?_Bad Expiration,', 'Errors?_Bad Expiration,Bad CVV,', 'Errors?_Bad Expiration,Insufficient Balance,',
            'Errors?_Bad Expiration,Technical Glitch,', 'Errors?_Bad PIN,', 'Errors?_Bad PIN,Insufficient Balance,', 
            'Errors?_Bad PIN,Technical Glitch,', 'Errors?_Bad Zipcode,', 'Errors?_Insufficient Balance,',
            'Errors?_Insufficient Balance,Technical Glitch,', 'Errors?_Technical Glitch,', 'Errors?_missing_value',
            'Year_Month_Day_Time', 'Amount']


### Construct the SQL CREATE statement

In [7]:
# build "CREATE TABLE" statement for postgresql
sql = "CREATE TABLE IF NOT EXISTS public.transactions ("

def get_dtype(d):
    if d == 'float64':
        return 'real'
    elif d == 'int64':
        return 'bigint'
    else: 
        return 'varchar'
    
for col, d in zip(tdf.columns, tdf.dtypes):
    sql += f'"{col.replace("?", "")}" {get_dtype(d)}, '
sql += ';'

In [8]:
import re
sql = re.sub(r', ;$', ';', sql)
sql

'CREATE TABLE IF NOT EXISTS public.transactions ("Is Fraud" bigint, "Merchant Name_x0_0" real, "Merchant Name_x0_1" real, "Merchant Name_x0_2" real, "Merchant Name_x0_3" real, "Merchant Name_x0_4" real, "Merchant Name_x0_5" real, "Merchant Name_x0_6" real, "Merchant Name_x0_7" real, "Merchant Name_x0_8" real, "Merchant Name_x0_9" real, "Merchant Name_x1_0" real, "Merchant Name_x1_1" real, "Merchant Name_x1_2" real, "Merchant Name_x1_3" real, "Merchant Name_x1_4" real, "Merchant Name_x1_5" real, "Merchant Name_x1_6" real, "Merchant Name_x1_7" real, "Merchant Name_x1_8" real, "Merchant Name_x1_9" real, "Merchant Name_x2_0" real, "Merchant Name_x2_1" real, "Merchant Name_x2_2" real, "Merchant Name_x2_3" real, "Merchant Name_x2_4" real, "Merchant Name_x2_5" real, "Merchant Name_x2_6" real, "Merchant Name_x2_7" real, "Merchant Name_x2_8" real, "Merchant Name_x2_9" real, "Merchant Name_x3_0" real, "Merchant Name_x3_1" real, "Merchant Name_x3_2" real, "Merchant Name_x3_3" real, "Merchant Name

### Transfer the data and fill the database

> create `init_transactions.sql` with the statement stored in `sql` (see above) and add the following line to the end of the file:

```\copy public.transactions FROM './preprocessed_transactions.csv' WITH (FORMAT csv, HEADER true, DELIMITER ',');```